In [7]:
from django_for_jupyter import init_django
init_django("arches")

import json
import pprint 
import uuid
import pandas as pd
from lxml import etree
from collections import defaultdict
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
from urllib.parse import urlparse

from arches.app.models.graph import Graph
from arches.app.models.concept import Concept
from arches.app.models.models import CardXNodeXWidget, Node, Resource2ResourceConstraint, FunctionXGraph, Value, GraphXPublishedGraph
from arches.app.utils.betterJSONSerializer import JSONSerializer, JSONDeserializer
#from operator import itemgetter
from arches.app.models import models

#from arches.app.models.system_settings import settings
from arches.app.utils.data_management.resource_graphs.exporter import get_graphs_for_export as get_json_graph
#from arches.app.utils.data_management.resource_graphs.exporter import create_mapping_configuration_file as get_mapping
#from arches.app.utils.data_management.resource_graphs.exporter import export as export_gephi
#from collections import OrderedDict
from django.core.serializers.json import DjangoJSONEncoder


#_model_uuid_list = [{"model_name": "Museological Item", "uuid": "fa952d08-dd27-11ed-9655-00163e71351b"}]

_model_uuid_list = [
    {"model_name": "Acquisition", "uuid": "35bd84a3-fc5c-4b9f-81ec-58a97eab9a97"},
    {"model_name": "Archival Unit", "uuid": "3b2193fc-04fb-11ee-9497-96a6d2455259"},
    {"model_name": "Bibliographical Item", "uuid": "daaa7b3c-0472-489b-a4b2-b7ba5854345c"},
    {"model_name": "Bibliographical Publication Expression", "uuid": "f4348d6a-04f9-11ee-9497-96a6d2455259"},
    #{"model_name": "Collection", "uuid": "cc4c1e91-68c8-492f-b68c-330acfd84f9c"},
    {"model_name": "Concept", "uuid": "74c87bc1-8a29-437a-88cc-3f49837dfc4e"},
    {"model_name": "Condition Assessment", "uuid": "eb25f6dd-74f5-4e39-8e96-36944d336714"},
    {"model_name": "Digital Object", "uuid": "ad7f933c-04fa-11ee-9497-96a6d2455259"},
    {"model_name": "Group", "uuid": "e53e0b75-d18e-46ce-8c50-865491d68d9c"},
    {"model_name": "Museological Item", "uuid": "fa952d08-dd27-11ed-9655-00163e71351b"},
    {"model_name": "Period", "uuid": "a4ae318d-f8f1-4b72-997d-1eba6e7ffa7d"},
    {"model_name": "Person", "uuid": "0d9ce288-e37c-11ed-9064-00163e71351b"},
    {"model_name": "Place", "uuid": "63c5daf1-3853-499b-8c9c-64fb149a2ae8"},
    {"model_name": "Product Instance", "uuid": "d54270fe-2d1f-4071-9410-71b098bb27f3"},
    #{"model_name": "Product Series", "uuid": "3fb48174-25d4-4659-8f7a-b673588a9766"},
    #{"model_name": "Product Type", "uuid": "03df26a0-cfe3-4be1-8931-08bd7c002a5f"}
                    
                  ]

ONTOLOGY_NAMESPACES = {#'http://my_namespace_here/': 'some_ns',
                    "http://purl.org/dc/elements/1.1/": "dc",
                    "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "RDF",
                    "http://www.w3.org/2001/XMLSchema#": "xsd",
                    "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
                    "http://www.cidoc-crm.org/cidoc-crm/CRMsci/": "CRMsci",
                    "http://www.cidoc-crm.org/cidoc-crm/": "crm",
                    "http://www.ics.forth.gr/isl/CRMarchaeo/": "CRMarchaeo",
                    "http://www.ics.forth.gr/isl/CRMdig/": "CRMdig",
                    "http://www.ics.forth.gr/isl/CRMgeo/": "CRMgeo",
                    "http://www.ics.forth.gr/isl/CRMinf/": "CRMinf",
                    "http://parthenos.d4science.org/CRMext/CRMpe.rdfs/": "CRMpe",
                    "https://takin.solutions/ontologies/CRMsurv/": "CRMsurv",
                    "https://linked.art/ns/terms/": "la",
                    "http://iflastandards.info/ns/fr/frbr/frbroo/": "frbr",    
    
}
model_uuid_list = {"model_name": "Bibliographical Item", "uuid": "daaa7b3c-0472-489b-a4b2-b7ba5854345c"}


In [8]:
def get_card_x_node_x_widget_data_for_export(resource_graph):
    cards_x_nodes_x_widgets = []
    nodeids = [node["nodeid"] for node in resource_graph["nodes"]]
    cards_x_nodes_x_widgets = CardXNodeXWidget.objects.filter(node_id__in=nodeids)
    return cards_x_nodes_x_widgets

def get_graph(graphid):        
    graph = get_json_graph([graphid])    
    source_graph = pd.DataFrame(graph['graph'])
    
    nodes_df = pd.DataFrame(graph['graph'][0]['nodes'])
    edges_df = pd.DataFrame(graph['graph'][0]['edges'])
    cards_df = pd.DataFrame(graph['graph'][0]['cards'])
    nodegroups_df = pd.DataFrame(graph['graph'][0]['nodegroups'])
    cards_x_nodes_x_widgets_df = pd.DataFrame(graph['graph'][0]['cards_x_nodes_x_widgets'])
        
    for idx, row in nodes_df.iterrows():
        node_id = row['nodeid']
        nodegroup_id = row['nodegroup_id']
        nodes_df.loc[idx, 'id'] = node_id
        nodes_df.loc[idx, 'elem_name'] = row['ontologyclass']
        nodes_df.loc[idx, 'elem_type'] = 'class'
        
        # top node
        if row['istopnode']:
            nodes_df.loc[idx, 'parentproperty'] = 'RDF'
            nodes_df.loc[idx, 'ontologyproperty'] = 'RDF'
            nodes_df.loc[idx, 'card_sortorder'] = -1
            nodes_df.loc[idx, 'parent_id'] = 'root'
            nodes_df.loc[idx, 'child_id'] = row['nodeid']
            nodes_df.loc[idx, 'visible'] = True
        
        # match edges to nodes
        matching_edge_row = edges_df[edges_df['rangenode_id'] == node_id]                
        if not matching_edge_row.empty:            
            nodes_df.loc[idx, 'edgeid'] = str(matching_edge_row['edgeid'].values[0])
            add_parent_id = edges_df[edges_df['rangenode_id'] == node_id]                
            if not add_parent_id.empty:
                nodes_df.loc[idx, 'parent_id'] = str(add_parent_id['edgeid'].values[0])
                #nodes_df.loc[idx, 'child_id'] = str(matching_edge_row['domainnode_id'].values[0])
            nodes_df.loc[idx, 'child_id'] = node_id
            
        # match cards
        if not row['istopnode']: ## the top node has no nodegroup_id
            matching_card_row = cards_df[cards_df['nodegroup_id'] == nodegroup_id]        
            if not matching_card_row.empty:            
                name = matching_card_row['name'].values[0]
                nodes_df.loc[idx, 'card_label'] = str(name)
                nodes_df.loc[idx, 'card_sortorder'] = matching_card_row['sortorder'].values[0]
                nodes_df.loc[idx, 'instructions'] = str(matching_card_row['instructions'].values[0])
                nodes_df.loc[idx, 'helptitle'] = str(matching_card_row['helptitle'].values[0])
                nodes_df.loc[idx, 'helptext'] = str(matching_card_row['helptext'].values[0])
                nodes_df.loc[idx, 'visible'] = str(matching_card_row['visible'].values[0])

                
                
        # match cards_x_nodes_x_widgets
        #matching_cards_x_nodes_x_widgets_row = cards_x_nodes_x_widgets_df[cards_x_nodes_x_widgets_df['node_id'] == node_id]
        #print(matching_cards_x_nodes_x_widgets_row)
        #if not matching_cards_x_nodes_x_widgets_row.empty:
        #    nodes_df.loc[idx, 'x_sortorder'] = matching_cards_x_nodes_x_widgets_row['sortorder'].values[0]
        #    nodes_df.loc[idx, 'x_card_label'] = str(matching_cards_x_nodes_x_widgets_row['label'].tolist()[0]['en'])
        #    nodes_df.loc[idx, 'x_card_id'] = matching_cards_x_nodes_x_widgets_row['card_id'].values[0]
        #    nodes_df.loc[idx, 'visible'] = matching_cards_x_nodes_x_widgets_row['visible'].values[0]
        
        # add edges to nodes
        add_edge_row = edges_df[edges_df['rangenode_id'] == node_id]                
        if not matching_edge_row.empty:   
            new_row_df = pd.DataFrame({
                'elem_type': 'property',
                'elem_name': str(matching_edge_row['ontologyproperty'].values[0]),
                'is_collector': False,
                'nodeid': '',
                'name': '',
                'description': '', 
                'istopnode': False,
                'ontologyclass': '',
                'datatype': 'property',
                'nodegroup_id': '',
                'graph_id': row['graph_id'],
                'config': '',
                'issearchable': '',
                'isrequired': True,
                'sortorder': '',
                'fieldname': '',
                'exportable': '',
                'alias': '',          
                'hascustomalias': '',
                'parentproperty': '',
                'id': str(add_edge_row['edgeid'].values[0]),
                'edgeid': str(add_edge_row['edgeid'].values[0]), 
                'parent_id': str(add_edge_row['domainnode_id'].values[0]),
                'child_id': str(add_edge_row['edgeid'].values[0]), #str(add_edge_row['rangenode_id'].values[0]),
                'card_label': '',
                'card_sortorder': 0,
                'visible': True,
                'ontologyproperty': ''
            }, index=[0])

            #nodes_df.loc[len(nodes_df.index)] = new_row_df
            nodes_df = pd.concat([nodes_df, new_row_df])   
            #print(len(new_row))    

    
    return nodes_df.sort_values(by=['card_sortorder'])
    #return new_row_df
#graph = get_graph('af04eac2-a131-11ed-a102-9cf387da2c40')
#graph.to_csv('sipc/out/temp.csv', index=False)
#graph

In [9]:
from collections import defaultdict
from typing import List, Dict


# Function to generate paths
def generate_paths(tree: Dict, node: str, path: List = []):
    path = path + [node]
    
    paths = [path]
    if node in tree:
        for child in tree[node]:
            paths.extend(generate_paths(tree, child, path))
    return paths

def make_uuid_paths(df):
    tree = df.groupby('parent_id')['child_id'].apply(list).to_dict()
    # Generate paths
    paths = generate_paths(tree, 'root')

    # Convert paths to DataFrame
    paths_df = pd.DataFrame([(path[-1], i, path) for i, path in enumerate(paths)], columns=['id', 'order', 'path'])

    # Merge paths_df with the existing df based on 'id'
    merged_df = pd.merge(graph, paths_df, on='id', how='left')
    #merged_df.to_csv('sipc/out/merged.csv', index=False)
    return merged_df


In [10]:
def add_namespace(path):

    for k,v in ONTOLOGY_NAMESPACES.items():
        if k in path:
            new_path = path.replace(k,f'{v}:')
            return new_path
    
def make_ns_paths(df):
    
    #df = df['card_sortorder'].astype(int)
    df = df.sort_values('order').reset_index(drop=True)
    path_list = []
    path_dict = {}
    n=0
    for idx, row in df.iterrows():
        full_path = row['path']        
        #print(full_path)
        for path in full_path:

            if path == 'root':
                ns_path = 'RDF'
            else:
                matching_row = df[df['id'] == path]
                ns_path = add_namespace(matching_row['elem_name'].values[0])
                #print(matching_row)
                if (n % 2) == 0:
                    pass
                    #print(matching_row['elem_name'])
                    #ns_path = add_namespace(matching_row['elem_name'].values[0])
                else:   
                    pass
                    #ns_path = add_namespace(matching_row['elem_name'].values[0])
            path_list.append(ns_path)
            ns_path = ''
            n+=1        
        n=0        
        df.loc[idx, 'ns_path'] = str(path_list)
        path_list = []
    return df

## Make rec-defs    

In [11]:

graph = get_graph(model_uuid_list['uuid'])
path_graph = make_uuid_paths(graph)
ns_path_graph = make_ns_paths(path_graph)

graph

,is_collector,nodeid,name,description,istopnode,ontologyclass,datatype,nodegroup_id,graph_id,config,...,ontologyproperty,card_sortorder,parent_id,child_id,visible,edgeid,card_label,instructions,helptitle,helptext
0,False,099f196c-df80-11ed-9232-00163e71351b,Bibliographical Item,,True,http://www.cidoc-crm.org/cidoc-crm/E22_Human-M...,semantic,None,daaa7b3c-0472-489b-a4b2-b7ba5854345c,{'en': ''},...,RDF,-1.0,root,099f196c-df80-11ed-9232-00163e71351b,True,NaN,NaN,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,099f196c-df80-11ed-9232-00163e71351b,6926ef62-ed8d-4121-ba84-2750849069fe,True,6926ef62-ed8d-4121-ba84-2750849069fe,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,35eb661e-ee5e-11ed-a1e6-00163e71351b,35ebbfc4-ee5e-11ed-a1e6-00163e71351b,True,35ebbfc4-ee5e-11ed-a1e6-00163e71351b,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,35eb661e-ee5e-11ed-a1e6-00163e71351b,35ebc2c6-ee5e-11ed-a1e6-00163e71351b,True,35ebc2c6-ee5e-11ed-a1e6-00163e71351b,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,35eb661e-ee5e-11ed-a1e6-00163e71351b,35ebc500-ee5e-11ed-a1e6-00163e71351b,True,35ebc500-ee5e-11ed-a1e6-00163e71351b,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,35eb6fce-ee5e-11ed-a1e6-00163e71351b,35ebc71c-ee5e-11ed-a1e6-00163e71351b,True,35ebc71c-ee5e-11ed-a1e6-00163e71351b,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,35eb6d76-ee5e-11ed-a1e6-00163e71351b,35ebc92e-ee5e-11ed-a1e6-00163e71351b,True,35ebc92e-ee5e-11ed-a1e6-00163e71351b,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,099f196c-df80-11ed-9232-00163e71351b,87bcd856-fdd0-43b2-8d40-dced7b5a7a50,True,87bcd856-fdd0-43b2-8d40-dced7b5a7a50,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,099f196c-df80-11ed-9232-00163e71351b,27d5bb7e-c8e8-47e2-aa85-576dc7fe2c0d,True,27d5bb7e-c8e8-47e2-aa85-576dc7fe2c0d,,NaN,NaN,NaN
0,False,,,,False,,property,,daaa7b3c-0472-489b-a4b2-b7ba5854345c,,...,,0.0,3d7e5e70-ed7f-11ed-9064-00163e71351b,3d7e9020-ed7f-11ed-9064-00163e71351b,True,3d7e9020-ed7f-11ed-9064-00163e71351b,,NaN,NaN,NaN


In [10]:
    #{"model_name": "Bibliographical Publication Expression", "uuid": "f4348d6a-04f9-11ee-9497-96a6d2455259"},
    #{"model_name": "Collection", "uuid": "cc4c1e91-68c8-492f-b68c-330acfd84f9c"},

test_graph = get_json_graph(['cc4c1e91-68c8-492f-b68c-330acfd84f9c'])
print(test_graph)

{'graph': []}
